In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/NLP/argmine/code/src-ipynb

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/NLP/argmine/code/src-ipynb


In [2]:
!pip install fast-pagerank;
!pip install py-rouge;
!pip install sentence_transformers;

     |████████████████████████████████| 56 kB 2.7 MB/s 
     |████████████████████████████████| 78 kB 3.6 MB/s 
     |████████████████████████████████| 3.5 MB 11.1 MB/s 
     |████████████████████████████████| 6.8 MB 30.9 MB/s 
     |████████████████████████████████| 1.2 MB 33.7 MB/s 
     |████████████████████████████████| 67 kB 5.6 MB/s 
     |████████████████████████████████| 596 kB 45.1 MB/s 
     |████████████████████████████████| 895 kB 38.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=505c67a550b967f6b00ae5da39589e3777adec8d59403b8c0ee197b6024230c2
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
import rouge
import os

In [7]:
evaluator = rouge.Rouge(metrics=['rouge-l'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

In [8]:
import json
import pandas as pd
import numpy as np
import spacy
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import itertools
import nltk
nltk.download('punkt')

from scipy import sparse
from fast_pagerank import pagerank
from fast_pagerank import pagerank_power


pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
from sentence_transformers import SentenceTransformer, InputExample, LoggingHandler, losses, models, util
import torch

In [10]:
train_args = pd.read_pickle('../../data/scores/train_args_with_scores.pkl')

In [11]:
train_topic_with_scored_kps = pd.read_pickle('../../data/scores/train_topic_keypoints_with_scores.pkl')

In [12]:
train_topics = train_args.groupby(['topic', 'stance']).agg({
    'sents_with_scores': lambda x: set([item for items in x for item in items]),
    'gt-kps' : lambda x: set([item for items in x for item in items]),
    #'gt-cand-kps': lambda x: set([item for items in x for item in items])
}).reset_index()

In [13]:
train_topics.columns

Index(['topic', 'stance', 'sents_with_scores', 'gt-kps'], dtype='object')

In [14]:
train_topics.head()

topic  ...                                                                                                                                                                                                                                                                                                                                                                                                                   gt-kps
0  Assisted suicide should be a criminal offence  ...                                                                                                                                                                                           {The terminally ill would benefit from assisted suicide, People should have the freedom to choose to end their life, Assisted suicide gives dignity to the person that wants to commit it, Assisted suicide reduces suffering}
1  Assisted suicide should be a criminal offence  ...                          {People do not have the right to self-harm/harm others, People who are ill do not have clear judgement and are in need of help, Assisted suicide violates the doctor's role, Assisted suicide allows people to solicit someone to die to their own benefit, Assisted suicide is akin to killing someone, Assisted suicide should not be allowed because many times people can still get better}
2                 Homeschooling should be banned  ...  {The home is a good learning environment, Homeschooling is often the best option for catering for the needs of exceptional/religious/ill/disabled students., Parents will have more ability to pay-attention/educate their child, Mainstream schools have a lot of violence/bullying, Homeschools can be personalized to the child's pace/needs, Parents should be permitted to choose the education of their children}
3                 Homeschooling should be banned  ...                                                                                                                                                                                                                   {Homeschools cannot be regulated/standardized, Mainstream schools are essential to develop social skills, Parents are not qualified as teachers, Mainstream schools are of higher educational quality}
4        The vow of celibacy should be abandoned  ...                                                                                                                                 {Religious experiences and traditions should be maintained, Removing celibacy could lead to sexually reckless acts, Celibacy is a strong religious belief, Priests choose to be celibates, Celibacy allows a priest to focus on God, Celibacy makes the priests to be perceived as holy}

[5 rows x 4 columns]

In [15]:
#Add to the candidate sentences from the topic, the topic key-points
train_topics['sents_with_scores'] = train_topics.apply(lambda row: list(row['sents_with_scores']) + train_topic_with_scored_kps[(train_topic_with_scored_kps.topic == row['topic']) & (train_topic_with_scored_kps.stance == row['stance'])]['scored_kps'].tolist()[0], axis=1)

In [16]:
model = SentenceTransformer('../../data/siamese-models/roberta-base-contrastive-10-epochs-2022-01-26_10-26-13')

In [23]:
def gen_match_matrix(model, sents, topic, min_match_score=0):
    sents1 = [topic + ' <SEP> ' + x for x in sents]
    sents1_embeddings = model.encode(sents1)
    sents_embeddings  = model.encode(sents)
    
    sim_matrix = cosine_similarity(sents1_embeddings, sents1_embeddings)
    super_threshold_indices = sim_matrix < min_match_score
    sim_matrix[super_threshold_indices] = 0

    return sim_matrix

def evalute_topic_by_rouge(row):
    ranked_sents = [x[0] for x in row['ranked_sents']][0:10]
    # gt_kps = [[x] for x in row['gt-kps']]
    gt_kps = [x for x in row['gt-kps']]

    r_l = []
    for a,b in itertools.product(gt_kps, ranked_sents):
        # print(a,b)
        # print(type(a), type(b))
        scores = evaluator.get_scores(a, b)
        r_l.append(scores['rouge-l']['f'])
    
    return np.mean(r_l)

def evalute_topic(row):
    ranked_sents = [x[0] for x in row['ranked_sents']]
    # ranked_sents = [x for x in row['filtered_sents']]
    gt_kps_ranks = []
    for real_kp in row['gt-kps']:
        if real_kp in ranked_sents:
            gt_kps_ranks.append(ranked_sents.index(real_kp)+1)
        else:
            print('kp is not there...')
            gt_kps_ranks.append(len(ranked_sents)+1)
            
    #print(gt_kps_ranks)
    return 1/(sum(gt_kps_ranks)/len(gt_kps_ranks)) if len(gt_kps_ranks) > 0 else 0

In [24]:
def apply_page_rank(row, p=0.85, min_quality_score=0.5, min_match_score=0.5, min_len=5, max_len=20):
    cand_sents = [x for x in row['sents_with_scores'] if x[1] > min_quality_score and len(x[0].split()) < max_len and len(x[0].split()) > min_len]
    if len(cand_sents) == 0:
        return []
        print('empty')
    cands, cands_qualities = zip(*cand_sents)
    #print(cands)
    #print(cands_qualities)
    cands_matching_mat = gen_match_matrix(model, cands, row['topic'], min_match_score=min_match_score)
    pr=pagerank(cands_matching_mat, personalize=np.array(cands_qualities), p=p) #it looks like modifying the initial probability doesn't help
    #pr=pagerank(cands_matching_mat, p=p)
    ranked_candidates = list(zip(cands, pr))
    return sorted(ranked_candidates, key=lambda x: -x[1])    

In [25]:
def filter_ranked_list(row, model, min_match=0.8):
    ranked_sents = [x[0] for x in row['ranked_sents']]
    filtered_sents = []
    while len(filtered_sents) < 5:
        filtered_sents = []
        for i, s in enumerate(ranked_sents):
            if len(filtered_sents) == 0:
                filtered_sents.append(s)
                print('FFFF ' , s)
            else:
                matching_scores = gen_match_matrix(model, [s]+filtered_sents, row['topic'])
                max_sim = np.max(matching_scores[0][1:])
                #print(matching_scores)
                if max_sim < min_match:
                    print('SSSS ' , s)
                    filtered_sents.append(s)
        
        min_match = min_match + 0.1
        print(min_match)

    return filtered_sents

In [26]:
scores = []
for min_match_score in [0.2, 0.4, 0.6, 0.8, 1.0]:
    for p in [0.2, 0.4, 0.8, 1.0]:
        for min_quality_score in [0.2, 0.5, 0.8]:
            train_topics['ranked_sents'] = train_topics.apply(lambda row: apply_page_rank(row, p=p, min_quality_score=min_quality_score, min_match_score=min_match_score), axis=1)
            train_topics['scores'] = train_topics.apply(lambda row: evalute_topic_by_rouge(row), axis=1)
            score = np.mean(train_topics.scores.tolist())
            scores.append([p, min_quality_score, min_match_score, score])

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/

In [27]:
scores_df = pd.DataFrame(scores, columns=['p', 'min-quality-score', 'min-match-score', 'score'])
scores_df.sort_values('score', ascending=False)

,p,min-quality-score,min-match-score,score
36,0.2,0.2,0.8,0.251731
37,0.2,0.5,0.8,0.251639
38,0.2,0.8,0.8,0.248984
39,0.4,0.2,0.8,0.247841
40,0.4,0.5,0.8,0.247568
49,0.2,0.5,1.0,0.247392
48,0.2,0.2,1.0,0.247392
54,0.8,0.2,1.0,0.247392
53,0.4,0.8,1.0,0.247392
52,0.4,0.5,1.0,0.247392


-------------

##### Apply on test:

In [28]:
# test_kps = pd.read_csv('../../data/key_points_our_test.csv')

test_kps = pd.read_csv('../../data/test_data/key_points_test.csv')

In [29]:
test_args = pd.read_pickle('../../data/scores/test_args_with_scores.pkl')

In [30]:
test_args.head()

,arg_id,argument,topic,stance,sents,sents_with_scores
0,arg_0_0,Routine child vaccinations isn't mandatory since children don't spread the virus,Routine child vaccinations should be mandatory,-1,[Routine child vaccinations isn't mandatory since children don't spread the virus],"[(Routine child vaccinations isn't mandatory since children don't spread the virus, 0.714194655418396)]"
1,arg_0_1,Routine child vaccinations should not be mandatory because children may not bear the side effects of it.,Routine child vaccinations should be mandatory,-1,[Routine child vaccinations should not be mandatory because children may not bear the side effects of it.],"[(Routine child vaccinations should not be mandatory because children may not bear the side effects of it., 0.8710675239562988)]"
2,arg_0_2,Routine child vaccinations should not be necessary as children can't catch the disease,Routine child vaccinations should be mandatory,-1,[Routine child vaccinations should not be necessary as children can't catch the disease],"[(Routine child vaccinations should not be necessary as children can't catch the disease, 0.8207754492759705)]"
3,arg_0_3,A vaccine that has not been sufficiently tested and without knowledge of side effects is not recommended for children,Routine child vaccinations should be mandatory,-1,[A vaccine that has not been sufficiently tested and without knowledge of side effects is not recommended for children],"[(A vaccine that has not been sufficiently tested and without knowledge of side effects is not recommended for children, 0.8951572775840759)]"
4,arg_0_4,"As long as vaccines are not free of side effects, it cannot make them mandatory for our children.",Routine child vaccinations should be mandatory,-1,"[As long as vaccines are not free of side effects, it cannot make them mandatory for our children.]","[(As long as vaccines are not free of side effects, it cannot make them mandatory for our children., 0.8620932698249817)]"


In [31]:
test_topics = test_args.groupby(['topic', 'stance']).agg({
    'sents_with_scores': lambda x: set([item for items in x for item in items])
}).reset_index()

In [32]:
test_topics['ranked_sents'] = test_topics.apply(lambda row: apply_page_rank(row, p=0.2, min_quality_score=0.8, min_match_score=0.8), axis=1)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseE

In [33]:
test_topics['ranked_kps'] = test_topics.apply(lambda row: filter_ranked_list(row, model, min_match=0.8), axis=1)

FFFF  vaccination should exclude children to avoid the side effects that can appear on them.
SSSS  When vaccines are mandatory, they can infringe on family religious choices
SSSS  Linking a measure as good as vaccination to coercive measures would cause serious harm
SSSS  child vaccinations is not mandatory because it may cost the country unnecessary funds.
SSSS  the poorest people may not be able to pay for the vaccine
SSSS  Vaccines contain heavy metals, they are toxic to children.
SSSS  Making child vaccination compulsory is not the the best way to obtain high vaccination rates
SSSS  vaccination of children should be limited as it affects child development
0.9
FFFF  Vaccinations are necessary to protect the health of the public as a whole and should be required by law.
SSSS  Child vaccination should be mandatory to provide decent healthcare equally
SSSS  Parents don't always know best and failure to vaccinate can be catastrophic for a child.
SSSS  childhood vaccination is necessary 

In [34]:
#adding gt-kps
test_topics['gt-kps']= test_topics.apply(lambda row: test_kps[(test_kps.topic==row['topic']) & (test_kps.stance==row['stance'])]['key_point'].tolist(), axis=1)

In [36]:
test_topics.head()

topic  ...                                                                                                                                                                                                                                                                                                              gt-kps
0                Routine child vaccinations should be mandatory  ...                                                                              [Routine child vaccinations, or their side effects, are dangerous, Mandatory vaccination contradicts basic rights, The parents and not the state should decide, Routine child vaccinations are not necessary to keep children healthy]
1                Routine child vaccinations should be mandatory  ...                                      [Routine child vaccinations are effective, Child vaccination saves lives, Routine child vaccinations are necessary to protect others, Routine child vaccinations should be mandatory to prevent virus/disease spreading, Children should not suffer from preventable diseases]
2  Social media platforms should be regulated by the government  ...      [Social media regulation is not effective, Social media regulation harms privacy, Social media regulation harm freedom of speech and other democratic rights, The government should not intervene in the affairs of a private company, Social media regulation can lead to political abuses by the government]
3  Social media platforms should be regulated by the government  ...  [Social media regulation protects the younger audiences, Social media regulation is necessary to deal with fake news, Social media regulation is beneficial to society at large, Social media regulation can help to deal with negative content, Social media regulation is required to deal with malicious users]
4                          The USA is a good country to live in  ...                                                   [The US has unfair health and education policies, The US has a problematic/divisive political system, The US has high taxation/high costs of living, The US is xenophobic/racist, The US has inequality/poverty, The US is unsafe, The US has a negative culture]

[5 rows x 6 columns]

In [37]:
test_topics[['topic', 'stance', 'ranked_kps', 'gt-kps']].to_pickle('../../data/pagerank-test-keypoints.pkl')

In [38]:
keypoints_ds = []
c = 0
for i, row in test_topics.iterrows():
    for kp in row['ranked_kps'][0:5]:
        keypoints_ds.append([c, row['topic'], row['stance'], kp])
        c+=1
        
keypoints_df = pd.DataFrame(keypoints_ds, columns=['key_point_id', 'topic', 'stance', 'key_point'])

In [39]:
keypoints_df.sample(n=5).head()

,key_point_id,topic,stance,key_point
10,10,Social media platforms should be regulated by the government,-1,the fact that the government regulates social media platforms undermines freedom of expression and thought
6,6,Routine child vaccinations should be mandatory,1,Child vaccination should be mandatory to provide decent healthcare equally
14,14,Social media platforms should be regulated by the government,-1,The only logical way to approach internet regulation is to do so from social consensus.
15,15,Social media platforms should be regulated by the government,1,Social media should be regulated by the government to prevent them from promoting dangerous activities and ideologies.
25,25,The USA is a good country to live in,1,"Bountiful food, good healthcare, stable government and personal citizens freedoms and rights."


In [40]:
keypoints_df.to_csv('../../data/pagerank-generated-test-keypoints.csv', index=False)

In [41]:
keypoints_df.topic.value_counts()

Routine child vaccinations should be mandatory                  10
Social media platforms should be regulated by the government    10
The USA is a good country to live in                            10
Name: topic, dtype: int64